# Original Data
The data provided have a lot of inconsistencies, in order to unpack its true potential, data clen up is necessary to address the data flaws to reveal the potential of the data and realize its true value. Initial analysis shows that the data have these issues; 
- Missing headers in the csv file
- Multiple variables are stored in one column
- Column data contains inconsistent unit values
- Missing values
- An empty row in the data
- Duplicate records in the data
- Non-ASCII characters
- Column headers are values and not variable names
- Encoding
+ Working with String .str
+ Split A Column Into Multiple Columns
+ Joining Two Columns
+ Removing Words,Expression
+ Searching For Strings and Expressions
+ Replacing Words

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
dataset = pd.read_csv("patient_heart_rate.csv")
dataset.head()

,1,Mickéy Mousé,56,70kgs,72,69,71,-,-.1,-.2
0,2.0,Donald Duck,34.0,154.89lbs,-,-,-,85,84,76
1,3.0,Mini Mouse,16.0,NaN,-,-,-,65,69,72
2,4.0,Scrooge McDuck,NaN,78kgs,78,79,72,-,-,-
3,5.0,Pink Panther,54.0,198.658lbs,-,-,-,69,NaN,75
4,6.0,Huey McDuck,52.0,189lbs,-,-,-,68,75,72


### Problem 1 – Missing headers in the csv file
If you look at the data file you notice that there is no header. Pandas is smart enough to figure out that the first line of the file is the header. We can still read the file if the csv file doesn’t have a header by manually providing the headers. Have a look at the below code.

In [2]:
import pandas as pd
column_names= ["Id", "Name", "Age", "Weight",'m0006','m0612','m1218','f0006','f0612','f1218']
dataset = pd.read_csv("patient_heart_rate.csv", names = column_names)
dataset.head()

,Id,Name,Age,Weight,m0006,m0612,m1218,f0006,f0612,f1218
0,1.0,Mickéy Mousé,56.0,70kgs,72,69,71,-,-,-
1,2.0,Donald Duck,34.0,154.89lbs,-,-,-,85,84,76
2,3.0,Mini Mouse,16.0,NaN,-,-,-,65,69,72
3,4.0,Scrooge McDuck,NaN,78kgs,78,79,72,-,-,-
4,5.0,Pink Panther,54.0,198.658lbs,-,-,-,69,NaN,75


### Problem 2 – Some columns are merged into a column 
In this dataset patient_heart_rate, the Name column contains two variables Lastname and Firstname(See Fig 3). To respect the tidy principles, we want to isolate each variable in a single column.

In [3]:
dataset.head()

,Id,Name,Age,Weight,m0006,m0612,m1218,f0006,f0612,f1218
0,1.0,Mickéy Mousé,56.0,70kgs,72,69,71,-,-,-
1,2.0,Donald Duck,34.0,154.89lbs,-,-,-,85,84,76
2,3.0,Mini Mouse,16.0,NaN,-,-,-,65,69,72
3,4.0,Scrooge McDuck,NaN,78kgs,78,79,72,-,-,-
4,5.0,Pink Panther,54.0,198.658lbs,-,-,-,69,NaN,75


In [4]:
dataset2 = dataset[['Firstname','Lastname']] = dataset['Name'].str.split(expand=True)
dataset2 = dataset.drop('Name', axis=1)
dataset2.head()

,Id,Age,Weight,m0006,m0612,m1218,f0006,f0612,f1218,Firstname,Lastname
0,1.0,56.0,70kgs,72,69,71,-,-,-,Mickéy,Mousé
1,2.0,34.0,154.89lbs,-,-,-,85,84,76,Donald,Duck
2,3.0,16.0,NaN,-,-,-,65,69,72,Mini,Mouse
3,4.0,NaN,78kgs,78,79,72,-,-,-,Scrooge,McDuck
4,5.0,54.0,198.658lbs,-,-,-,69,NaN,75,Pink,Panther


### Problem 3 – Column data contains inconsistent unit values
If you observe in the dataset, the way the measurement unit was written for Weight column is not consistent. There are rows that contain data in kgs unit measure while there are cases that it is in lbs. As part of the  data preprocessing, we need to remove any inconsistency and bring the data together in one consistent unit measure

In [5]:
weight = dataset2['Weight']
 
for i in range (0 ,len(weight)):    
    x= str(weight[i])

    if 'lbs' in x[-3:]:            #Remove the lbs from the value
        lbs = x[:-3:]
        float_lbs = float(lbs)     #Convert string to float
        kgs = int(float_lbs/2.206) #Covert to kgs and store as int
        kgs = str(kgs)+"kgs"       #Convert back to string
        weight[i]= kgs
dataset2.head()

,Id,Age,Weight,m0006,m0612,m1218,f0006,f0612,f1218,Firstname,Lastname
0,1.0,56.0,70kgs,72,69,71,-,-,-,Mickéy,Mousé
1,2.0,34.0,70kgs,-,-,-,85,84,76,Donald,Duck
2,3.0,16.0,NaN,-,-,-,65,69,72,Mini,Mouse
3,4.0,NaN,78kgs,78,79,72,-,-,-,Scrooge,McDuck
4,5.0,54.0,90kgs,-,-,-,69,NaN,75,Pink,Panther


### Problem 4 – Missing values
There are few missing values in the Age, Weight and Heart Rate columns of the dataset and it is quite common that some values could be missing from dataset. This typically means that a piece of information was simply not collected and corrupted during the process of data retrieval. Regardless of the reason, It is higly encouraged to have a discussion with subject matter experts (SME) how to treat the missing data. Typically missing values could be treated as follows;
a. Deletion: Remove records with missing values.
b. Dummy substitution: Replace missing values with a dummy but valid value: e.g.: 0 for numerical values.
c. Substitution: (Mean, Median)
d. Frequent substitution: Replace the missing values with the most frequent item

As part of proess improvement, it is advisable to improve the data collection process. 

In [6]:
dataset.isna().any()

Id           True
Name         True
Age          True
Weight       True
m0006        True
m0612        True
m1218        True
f0006        True
f0612        True
f1218        True
Firstname    True
Lastname     True
dtype: bool

In [7]:
dataset2.isna().sum()

Id           1
Age          2
Weight       2
m0006        1
m0612        1
m1218        1
f0006        1
f0612        2
f1218        1
Firstname    1
Lastname     1
dtype: int64

### Problem 5 – An empty row in the data
If you observe in the dataset, you can see that there is a Empty row. If you read this file with Pandas library, and look at the content of your dataframe, you have a empty row that has been filled with NaNs.See Fig 7

There is no option to ignore the row in the function read_csv(), so we need to handle this using dropna() method as shown in below code.

In [8]:
dataset2

,Id,Age,Weight,m0006,m0612,m1218,f0006,f0612,f1218,Firstname,Lastname
0,1.0,56.0,70kgs,72,69,71,-,-,-,Mickéy,Mousé
1,2.0,34.0,70kgs,-,-,-,85,84,76,Donald,Duck
2,3.0,16.0,NaN,-,-,-,65,69,72,Mini,Mouse
3,4.0,NaN,78kgs,78,79,72,-,-,-,Scrooge,McDuck
4,5.0,54.0,90kgs,-,-,-,69,NaN,75,Pink,Panther
5,6.0,52.0,85kgs,-,-,-,68,75,72,Huey,McDuck
6,7.0,19.0,56kgs,-,-,-,71,78,75,Dewey,McDuck
7,8.0,32.0,78kgs,78,76,75,-,-,-,Scööpy,Doo
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9.0,52.0,85kgs,-,-,-,68,75,72,Huey,McDuck


In [9]:
dataset2.dropna(how="all", inplace=True)
dataset2

,Id,Age,Weight,m0006,m0612,m1218,f0006,f0612,f1218,Firstname,Lastname
0,1.0,56.0,70kgs,72,69,71,-,-,-,Mickéy,Mousé
1,2.0,34.0,70kgs,-,-,-,85,84,76,Donald,Duck
2,3.0,16.0,NaN,-,-,-,65,69,72,Mini,Mouse
3,4.0,NaN,78kgs,78,79,72,-,-,-,Scrooge,McDuck
4,5.0,54.0,90kgs,-,-,-,69,NaN,75,Pink,Panther
5,6.0,52.0,85kgs,-,-,-,68,75,72,Huey,McDuck
6,7.0,19.0,56kgs,-,-,-,71,78,75,Dewey,McDuck
7,8.0,32.0,78kgs,78,76,75,-,-,-,Scööpy,Doo
9,9.0,52.0,85kgs,-,-,-,68,75,72,Huey,McDuck
10,10.0,12.0,45kgs,-,-,-,92,95,87,Louie,McDuck


### Problem 6 – Duplicate records in the data
Sometimes, in the dataset there may be some duplicate records. In our dataset there is a record which is repeated in 6th and 9th rows. See Fig 9 for duplicate records.

In [10]:
dupe = dataset2[dataset2.duplicated(['Firstname','Lastname'])]
print(dupe)

    Id   Age Weight m0006 m0612 m1218 f0006 f0612 f1218 Firstname Lastname
9  9.0  52.0  85kgs     -     -     -    68    75    72      Huey   McDuck


In [11]:
dataset2[(dataset2['Firstname'] == 'Huey') & (dataset2['Lastname'] == 'McDuck')] 

,Id,Age,Weight,m0006,m0612,m1218,f0006,f0612,f1218,Firstname,Lastname
5,6.0,52.0,85kgs,-,-,-,68,75,72,Huey,McDuck
9,9.0,52.0,85kgs,-,-,-,68,75,72,Huey,McDuck


In [12]:
dataset2

,Id,Age,Weight,m0006,m0612,m1218,f0006,f0612,f1218,Firstname,Lastname
0,1.0,56.0,70kgs,72,69,71,-,-,-,Mickéy,Mousé
1,2.0,34.0,70kgs,-,-,-,85,84,76,Donald,Duck
2,3.0,16.0,NaN,-,-,-,65,69,72,Mini,Mouse
3,4.0,NaN,78kgs,78,79,72,-,-,-,Scrooge,McDuck
4,5.0,54.0,90kgs,-,-,-,69,NaN,75,Pink,Panther
5,6.0,52.0,85kgs,-,-,-,68,75,72,Huey,McDuck
6,7.0,19.0,56kgs,-,-,-,71,78,75,Dewey,McDuck
7,8.0,32.0,78kgs,78,76,75,-,-,-,Scööpy,Doo
9,9.0,52.0,85kgs,-,-,-,68,75,72,Huey,McDuck
10,10.0,12.0,45kgs,-,-,-,92,95,87,Louie,McDuck


In [13]:
dataset2 = dataset2.drop_duplicates(subset=['Firstname','Lastname'])
dataset2

,Id,Age,Weight,m0006,m0612,m1218,f0006,f0612,f1218,Firstname,Lastname
0,1.0,56.0,70kgs,72,69,71,-,-,-,Mickéy,Mousé
1,2.0,34.0,70kgs,-,-,-,85,84,76,Donald,Duck
2,3.0,16.0,NaN,-,-,-,65,69,72,Mini,Mouse
3,4.0,NaN,78kgs,78,79,72,-,-,-,Scrooge,McDuck
4,5.0,54.0,90kgs,-,-,-,69,NaN,75,Pink,Panther
5,6.0,52.0,85kgs,-,-,-,68,75,72,Huey,McDuck
6,7.0,19.0,56kgs,-,-,-,71,78,75,Dewey,McDuck
7,8.0,32.0,78kgs,78,76,75,-,-,-,Scööpy,Doo
10,10.0,12.0,45kgs,-,-,-,92,95,87,Louie,McDuck


### Problem 7 – Non-ASCII characters
There are few non-ASCII characters in the Firstname and Lastname columns in the data. See Fig 11 below

In [14]:
dataset2.Firstname.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
dataset2.Lastname.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
print (dataset2)

      Id   Age Weight m0006 m0612 m1218 f0006 f0612 f1218 Firstname Lastname
0    1.0  56.0  70kgs    72    69    71     -     -     -     Micky     Mous
1    2.0  34.0  70kgs     -     -     -    85    84    76    Donald     Duck
2    3.0  16.0    NaN     -     -     -    65    69    72      Mini    Mouse
3    4.0   NaN  78kgs    78    79    72     -     -     -   Scrooge   McDuck
4    5.0  54.0  90kgs     -     -     -    69   NaN    75      Pink  Panther
5    6.0  52.0  85kgs     -     -     -    68    75    72      Huey   McDuck
6    7.0  19.0  56kgs     -     -     -    71    78    75     Dewey   McDuck
7    8.0  32.0  78kgs    78    76    75     -     -     -      Scpy      Doo
10  10.0  12.0  45kgs     -     -     -    92    95    87     Louie   McDuck


### Problem 8 – Column headers are values and not variable names
The columns headers are composed of the values for Sex and Hour range variables for each individual row. Corresponding to each ‘m’ column for males, there is also an ‘f’ column for females, f0006, f0612 and so on as shown in the Fig 13.


In [15]:
dataset2

,Id,Age,Weight,m0006,m0612,m1218,f0006,f0612,f1218,Firstname,Lastname
0,1.0,56.0,70kgs,72,69,71,-,-,-,Micky,Mous
1,2.0,34.0,70kgs,-,-,-,85,84,76,Donald,Duck
2,3.0,16.0,NaN,-,-,-,65,69,72,Mini,Mouse
3,4.0,NaN,78kgs,78,79,72,-,-,-,Scrooge,McDuck
4,5.0,54.0,90kgs,-,-,-,69,NaN,75,Pink,Panther
5,6.0,52.0,85kgs,-,-,-,68,75,72,Huey,McDuck
6,7.0,19.0,56kgs,-,-,-,71,78,75,Dewey,McDuck
7,8.0,32.0,78kgs,78,76,75,-,-,-,Scpy,Doo
10,10.0,12.0,45kgs,-,-,-,92,95,87,Louie,McDuck


In [16]:
#Melt the Sex + time range columns in single column
dataset2 = pd.melt(dataset2,id_vars=['Id','Age','Weight','Firstname','Lastname'], value_name="PulseRate",var_name="sex_and_time").sort_values(['Id','Age','Weight','Firstname','Lastname'])
 
# Extract Sex, Hour lower bound and Hour upper bound group
tmp_df = dataset2["sex_and_time"].str.extract("(\D)(\d+)(\d{2})",expand=True)
 
# Name columns
tmp_df.columns = ["Sex", "hours_lower", "hours_upper"]
 
# Create Time column based on "hours_lower" and "hours_upper" columns
tmp_df["Time"] = tmp_df["hours_lower"] + "-" + tmp_df["hours_upper"]
 
# Merge 
dataset2 = pd.concat([dataset2, tmp_df], axis=1)
 
# Drop unnecessary columns and rows
dataset2 = dataset2.drop(['sex_and_time','hours_lower','hours_upper'], axis=1)
dataset2 = dataset2.dropna()
dataset2.to_csv('outputcleanup.csv',index=False)
print (dataset2.head())

     Id   Age Weight Firstname Lastname PulseRate Sex   Time
0   1.0  56.0  70kgs     Micky     Mous        72   m  00-06
9   1.0  56.0  70kgs     Micky     Mous        69   m  06-12
18  1.0  56.0  70kgs     Micky     Mous        71   m  12-18
27  1.0  56.0  70kgs     Micky     Mous         -   f  00-06
36  1.0  56.0  70kgs     Micky     Mous         -   f  06-12


# Other Data Cleaning Practical Examples

In [148]:
import pandas as pd
dataset = pd.read_csv('dataset.csv')
dataset.head(1)

,id,Full Name,Date of Birth,email,gender,Job,Company,Country,Salary,Phone,Quote,Income,Income.1
0,1,Salim Pound,2/12/2018,spound0@spotify.com,Male,Editor,Gigashots,Israel,19QtDp5HkyuZZqwwqzYPSSLuA7MXFCwfKL,610-741-3882,Compatible discrete leverage,46476.63808,$46476.6380809961


## Some good practices to follow
- improve column consistency
- naming conventions
- replace column names if needed

In [149]:
dataset.columns = dataset.columns.str.lower()
dataset.head(1)

,id,full name,date of birth,email,gender,job,company,country,salary,phone,quote,income,income.1
0,1,Salim Pound,2/12/2018,spound0@spotify.com,Male,Editor,Gigashots,Israel,19QtDp5HkyuZZqwwqzYPSSLuA7MXFCwfKL,610-741-3882,Compatible discrete leverage,46476.63808,$46476.6380809961


In [150]:
dataset.rename(columns={'full name':'full_name','date of birth':'date_of_birth'}).head(1)

,id,full_name,date_of_birth,email,gender,job,company,country,salary,phone,quote,income,income.1
0,1,Salim Pound,2/12/2018,spound0@spotify.com,Male,Editor,Gigashots,Israel,19QtDp5HkyuZZqwwqzYPSSLuA7MXFCwfKL,610-741-3882,Compatible discrete leverage,46476.63808,$46476.6380809961


In [151]:
# Rename and Replace Column Names
dataset.rename(columns={'full name':'full_name','date of birth':'date_of_birth'},inplace=True)
dataset.head(1)

,id,full_name,date_of_birth,email,gender,job,company,country,salary,phone,quote,income,income.1
0,1,Salim Pound,2/12/2018,spound0@spotify.com,Male,Editor,Gigashots,Israel,19QtDp5HkyuZZqwwqzYPSSLuA7MXFCwfKL,610-741-3882,Compatible discrete leverage,46476.63808,$46476.6380809961


## Splitting a column in to multiple columns
- The full name can be used as an example. Say we have to find out the most popular name on the list

### Use Split function (See also Sample#2 and Expand function)

In [152]:
dataset.full_name[:5]

0         Salim Pound
1      Madelene Lutas
2    Dolli Swanbourne
3     Graeme Croisier
4      Jillana Physic
Name: full_name, dtype: object

In [153]:
dataset.full_name.str.split(" ")
dataset.full_name[:5]

0         Salim Pound
1      Madelene Lutas
2    Dolli Swanbourne
3     Graeme Croisier
4      Jillana Physic
Name: full_name, dtype: object

### Check if split is okay. Put in in variable
f_name = dataset.full_name.str.split(" ").str.get(0)
l_name = dataset.full_name.str.split(" ").str.get(1)

In [154]:
f_name = dataset.full_name.str.split(" ").str.get(0)

In [155]:
f_name[:5]

0       Salim
1    Madelene
2       Dolli
3      Graeme
4     Jillana
Name: full_name, dtype: object

In [156]:
l_name = dataset.full_name.str.split(" ").str.get(1)

In [157]:
l_name[:5]

0         Pound
1         Lutas
2    Swanbourne
3      Croisier
4        Physic
Name: full_name, dtype: object

### Update the DataFrame with the correct variables

In [158]:
dataset['last_name'] = dataset.full_name.str.split(" ").str.get(1)
dataset['first_name'] = dataset.full_name.str.split(" ").str.get(0)

In [159]:
dataset.head(2)

,id,full_name,date_of_birth,email,gender,job,company,country,salary,phone,quote,income,income.1,last_name,first_name
0,1,Salim Pound,2/12/2018,spound0@spotify.com,Male,Editor,Gigashots,Israel,19QtDp5HkyuZZqwwqzYPSSLuA7MXFCwfKL,610-741-3882,Compatible discrete leverage,46476.63808,$46476.6380809961,Pound,Salim
1,2,Madelene Lutas,9/20/2018,mlutas1@si.edu,Female,VP Marketing,Wikivu,Mongolia,17ADLE8HiTj7m1tTbzboLDn7eiZ4TjetzZ,221-725-8860,Grass-roots real-time algorithm,54174.68795,$54174.6879482406,Lutas,Madelene


### Use Expand function (See Also Sample#2 and Split functions)

In [160]:
dataset.full_name[:5]

0         Salim Pound
1      Madelene Lutas
2    Dolli Swanbourne
3     Graeme Croisier
4      Jillana Physic
Name: full_name, dtype: object

In [161]:
dataset.full_name.str.split(" ",expand=True)[:5]

,0,1,2,3
0,Salim,Pound,None,None
1,Madelene,Lutas,None,None
2,Dolli,Swanbourne,None,None
3,Graeme,Croisier,None,None
4,Jillana,Physic,None,None


In [162]:
dataset.full_name.str.split(" ",n=1,expand=True)[:5]

,0,1
0,Salim,Pound
1,Madelene,Lutas
2,Dolli,Swanbourne
3,Graeme,Croisier
4,Jillana,Physic


In [163]:
dataset.head(5)

,id,full_name,date_of_birth,email,gender,job,company,country,salary,phone,quote,income,income.1,last_name,first_name
0,1,Salim Pound,2/12/2018,spound0@spotify.com,Male,Editor,Gigashots,Israel,19QtDp5HkyuZZqwwqzYPSSLuA7MXFCwfKL,610-741-3882,Compatible discrete leverage,46476.638080,$46476.6380809961,Pound,Salim
1,2,Madelene Lutas,9/20/2018,mlutas1@si.edu,Female,VP Marketing,Wikivu,Mongolia,17ADLE8HiTj7m1tTbzboLDn7eiZ4TjetzZ,221-725-8860,Grass-roots real-time algorithm,54174.687950,$54174.6879482406,Lutas,Madelene
2,3,Dolli Swanbourne,11/18/2018,dswanbourne2@istockphoto.com,Female,Associate Professor,Dabfeed,Vietnam,16ThVxH72YAXWiEBeBZB1WYtXcfJMwUtoB,303-720-8280,Innovative system-worthy definition,5449.873348,$5449.87334818567,Swanbourne,Dolli
3,4,Graeme Croisier,9/29/2018,gcroisier3@parallels.com,Male,Mechanical Systems Engineer,Trudoo,Bosnia and Herzegovina,1JLUVwmjHvxgh53C7Afie3pWbjrYxV9Fnx,673-794-9917,Customer-focused didactic extranet,46784.630880,$46784.6308786279,Croisier,Graeme
4,5,Jillana Physic,4/19/2018,jphysic4@yellowbook.com,Female,Account Executive,Buzzshare,China,1DaEUNoAWL68pnvThSM5eBYgQMTv9mKyFG,709-146-6263,Progressive dynamic model,57486.495560,$57486.4955595568,Physic,Jillana


In [164]:
dataset = dataset.drop('full_name', axis=1)

In [129]:
dataset.head(3)

,id,date_of_birth,email,gender,job,company,country,salary,phone,quote,income,income.1,last_name,first_name
0,1,2/12/2018,spound0@spotify.com,Male,Editor,Gigashots,Israel,19QtDp5HkyuZZqwwqzYPSSLuA7MXFCwfKL,610-741-3882,Compatible discrete leverage,46476.638080,$46476.6380809961,Pound,Salim
1,2,9/20/2018,mlutas1@si.edu,Female,VP Marketing,Wikivu,Mongolia,17ADLE8HiTj7m1tTbzboLDn7eiZ4TjetzZ,221-725-8860,Grass-roots real-time algorithm,54174.687950,$54174.6879482406,Lutas,Madelene
2,3,11/18/2018,dswanbourne2@istockphoto.com,Female,Associate Professor,Dabfeed,Vietnam,16ThVxH72YAXWiEBeBZB1WYtXcfJMwUtoB,303-720-8280,Innovative system-worthy definition,5449.873348,$5449.87334818567,Swanbourne,Dolli


## Finding string and replace it with correct values

In [168]:
dataset['income.1'][:3]

0    $46476.6380809961
1    $54174.6879482406
2    $5449.87334818567
Name: income.1, dtype: object

In [170]:
dataset['income.1'].str.replace("$"," ")[:3]

0     46476.6380809961
1     54174.6879482406
2     5449.87334818567
Name: income.1, dtype: object

In [101]:
dataset['income.1'].dtype

dtype('O')

In [103]:
dataset['income.1'].str.replace("$"," ")[:5]

0     46476.6380809961
1     54174.6879482406
2     5449.87334818567
3     46784.6308786279
4     57486.4955595568
Name: income.1, dtype: object

In [ ]:
'''
time
date
regex
''''''